# L2HMC for $U(1)$ Gauge Model 

## Imports

In [ ]:
import os
import sys
import time
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

#from l2hmc_eager import dynamics_eager as _l2hmc
#from l2hmc_eager import gauge_dynamics_eager as l2hmc
#from l2hmc_eager.neural_nets import *
#from utils.distributions import GMM, gen_ring
#from utils.jacobian import _map, jacobian
from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)
#from u1_model_eager import *

%autoreload 2

In [ ]:
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
from l2hmc_eager import gauge_dynamics_eager as gde

In [ ]:
tf.enable_eager_execution()
tfe = tf.contrib.eager

In [ ]:
from lattice.gauge_lattice import u1_plaq_exact

In [ ]:
from u1_model_eager import GaugeModelEager, train_one_iter
#from gauge_model import GaugeModel, graph_step, train_one_iter

In [ ]:
from utils.gauge_model_helpers import plot_run_data

In [ ]:
import utils.gauge_model_helpers as helpers

In [ ]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

In [ ]:
def graph_step(dynamics, samples, optimizer, loss_fn, 
               params,  global_step=None, hmc=False):
    """Perform a single training step using the compiled tensorflow graph.
    
    NOTE: 
        To be defunnable, the function cannot return an Operation, so the above
        function is used for defun or eager, and this function is used in graph to be
        able to run the gradient updates.
    """
    clip_value = params.get('clip_value', 100)
    loss, samples_out, accept_prob, grads = gde.loss_and_grads(
        dynamics=dynamics,
        x=samples,
        params=params,
        loss_fn=loss_fn,
        hmc=hmc
    )
    
    grads, _ = tf.clip_by_global_norm(grads, clip_value)
    train_op = optimizer.apply_gradients(
        zip(grads, dynamics.trainable_variables), global_step=global_step
    )
    
    return train_op, loss, samples_out, accept_prob, grads

## Use GaugeModelEager for training L2HMC on $U(1)$ gauge lattice

### Define params

In [ ]:
params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 8.,
    'num_samples': 4,
    'num_steps': 5,
    'eps': 0.2,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-3,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.96,
    'train_steps': 1000,
    'record_loss_every': 50,
    'data_steps': 1,
    'save_steps': 50,
    'print_steps': 5,
    'logging_steps': 50,
    'clip_value': 100,
    'rand': False,
    'metric': 'l2',
}
tf.reset_default_graph()

### Using gauge_model.GaugeModel (compiled graph)

Seems to have the same problem as before where the training time / step increases as training progresses. I think this is due to the graph constantly creating new `tf.reshape` operations each time a batch is fed through. Need to test further.

In [ ]:
model_conv = GaugeModel(params=params,
                        conv_net=True,
                        hmc=False,
                        log_dir=None,
                        restore=False)

In [ ]:
model_conv.logging_steps = 50
model_conv.data_steps = 1 
model_conv.save_steps = 50

In [ ]:
model_conv.train(500)

In [ ]:
tf.enable_resource_variables()
tf.reset_default_graph()
print("Building graph...")
model_conv = GaugeModelEager(params=params,
                             conv_net=True,
                             hmc=False,
                             log_dir=None,
                             restore=False,
                             defun=False)
x = tf.placeholder(tf.float32, shape=model_conv.samples.shape)
samples_np = np.array(model_conv.lattice.samples, dtype=np.float32)
#x = np.array(model_conv.lattice.samples, dtype=np.float32)
loss, _, _ = gde.compute_loss(model_conv.dynamics, x, model_conv.params)
train_op, loss, samples_out, accept_prob, _ = graph_step(
    dynamics=model_conv.dynamics,
    samples=x,
    optimizer=model_conv.optimizer,
    loss_fn=gde.compute_loss,
    params=model_conv.params,
    global_step=model_conv.global_step,
    hmc=model_conv.hmc
)
session_conf = tf.ConfigProto()
print("done.")

In [ ]:
#with tf.Session(config=session_conf) as sess:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
print("Performing warmup...")
for _ in range(1):
    _, _ = sess.run([train_op, loss], feed_dict={x: samples_np})
print("done.")
print("Training...")
start_time = time.time()
for i in range(10):
    _, loss_np, samples_np, accept_prob_np = sess.run(
        [train_op,
         loss,
         samples_out,
         accept_prob],
        feed_dict={x: samples_np}
    )
    print(f'step: {i}, loss: {loss_np}')
#x_, v_, x_accept_prob, x_out = model_conv.dynamics.apply_transition(x)

In [ ]:
tf.reset_default_graph()
      with tf.Graph().as_default():
        energy_fn, _, _ = l2hmc.get_scg_energy_fn()
        x = tf.random_normal([hparams.n_samples, hparams.x_dim],
                             dtype=tf.float32)
        dynamics = l2hmc.Dynamics(
            x_dim=hparams.x_dim,
            minus_loglikelihood_fn=energy_fn,
            n_steps=hparams.n_steps,
            eps=hparams.eps)
        loss, _, _ = l2hmc.compute_loss(dynamics, x)
        optimizer = tf.train.AdamOptimizer(learning_rate=hparams.learning_rate)
        train_op, loss, _ = graph_step(dynamics, optimizer, x)
        # Single thread; fairer comparison against eager
        session_conf = tf.ConfigProto(inter_op_parallelism_threads=1)
        with tf.Session(config=session_conf) as sess:
          sess.run(tf.global_variables_initializer())

          # Warmup to reduce initialization effect when timing
          for _ in range(hparams.n_warmup_iters):
            _, _ = sess.run([train_op, loss])

          # Training
          start_time = time.time()
          for i in range(hparams.n_iters):
            _, loss_np = sess.run([train_op, loss])
            print("Iteration %d: loss %.4f" % (i, loss_np))

### Using tf.contrib.eager execution with 'defun' for compiling ops to graph

#### Create model

##### (Generic) HMC Model

In [ ]:
model_hmc = GaugeModelEager(params=params,
                            conv_net=False,
                            hmc=True,
                            log_dir=None,
                            restore=False,
                            defun=True)
# 1m 22.9s

In [ ]:
observables_hmc = model_hmc.calc_observables(model_hmc.samples, 
                                             _print=True, 
                                             _write=True)
total_actions, avg_plaquettes, top_charges = observables_hmc

In [ ]:
model_hmc.train(10, keep_samples=False)

In [ ]:
steps = np.arange(len(model_hmc.data['average_plaquettes_arr']))
_ = plot_run_data(model_hmc.data, model_hmc.params, 
                  steps, model_hmc.figs_dir, skip_steps=1)

##### ConvNet L2HMC Model

In [ ]:
tf.reset_default_graph()

In [ ]:
model_conv_defun = GaugeModelEager(params=params,
                                   conv_net=True,
                                   hmc=False,
                                   log_dir=None,
                                   restore=False,
                                   defun=True)

In [ ]:
#params['conv_net'] = True
model_conv = GaugeModelEager(params=params,
                             conv_net=True,
                             hmc=False,
                             log_dir=None,
                             restore=False,
                             defun=False)
#model_conv._restore_model(model_conv.log_dir)
#model_conv.params

In [ ]:
observables_conv = model_conv.calc_observables(model_conv.samples, update=True) 
helpers.print_run_data(model_conv.data, header=True)
helpers.write_run_data(model_conv.files['run_info_file'], model_conv.data)

In [ ]:
model_conv.train(5)

In [ ]:
model_conv_defun.train(5)

In [ ]:
model_conv.train(500)
# 3h 50m 20s with defun=False, including aux var z, 5 steps eps 0.2

In [ ]:
steps_arr = [0]
steps_arr.extend(model_conv.steps_arr)

_ = plot_run_data(model_conv.data, model_conv.params, steps_arr, 
                  model_conv.figs_dir, skip_steps=1)

In [ ]:
_ = plot_run_data(model_conv.data, model_conv.params, steps_arr, 
                  model_conv.figs_dir, skip_steps=1)

##### FullyConnected (GenericNet) L2HMC Model

In [ ]:
model_fc = GaugeModelEager(params=params,
                           conv_net=False,
                           hmc=False,
                           log_dir=None,
                           restore=False,
                           defun=True)
#model_fc.params

In [ ]:
observables_fc = model_fc.calc_observables(model_fc.samples, update=True) 
helpers.print_run_data(model_fc.data, header=True)
helpers.write_run_data(model_fc.files['run_info_file'], model_fc.data)

In [ ]:
model_fc.train(500)

### Evaluate model (generate samples to measure Autocorrelation)

#### Using CONV NET model: `model_conv`

In [ ]:
#samples_conv = np.array(
#    model_conv.lattice.samples.reshape((model_conv.batch_size, -1)),
#    dtype=np.float32
#)
apply_transition_conv = tfe.defun(model_conv.dynamics.apply_transition)

In [ ]:
samples_conv = tf.random_normal(shape=model_conv.samples.shape)
samples_history_conv = []
actions_history_conv = []
avg_plaquettes_history_conv = []
top_charges_history_conv = []

In [ ]:
for i in range(500):
    #samples_history_conv.append(samples_conv.numpy())
    t0 = time.time()
    #_, _, _, samples_conv = apply_transition_conv(samples_conv)
    _, _, _, samples_conv1 = model_conv1.dynamics.apply_transition(samples_conv1)
    observables_conv = np.array(
        model_conv.lattice.calc_plaq_observables(samples_conv1)
    ).T
    actions_history_conv.append(observables_conv[0])
    avg_plaquettes_history_conv.append(observables_conv[1])
    top_charges_history_conv.append(observables_conv[2])
    step_time = (time.time() - t0) / (model_conv.num_steps * model_conv.batch_size)
    print(f'step: {i}  time/step: {step_time:^6.4g} '
          f'top_charge: {np.mean(observables_conv[2]):^6.4g} '
          f'avg_plaq: {np.mean(observables_conv[1]):^6.4g}')

In [ ]:
#samples_history_conv = np.array(samples_history_conv)
actions_history_conv = np.array(actions_history_conv)
avg_plaquettes_history_conv = np.array(avg_plaquettes_history_conv)
top_charges_history_conv = np.array(top_charges_history_conv)

In [ ]:
top_charges_autocorr0_conv = autocorr(top_charges_history_conv[:, 0])
top_charges_autocorr1_conv = autocorr(top_charges_history_conv[:, 1])

top_charges_autocorr_conv = (top_charges_autocorr0_conv 
                            + top_charges_autocorr1_conv) / 2

In [ ]:
avg_plaquettes_autocorr0_conv = autocorr(avg_plaquettes_history_conv[:, 0])
avg_plaquettes_autocorr1_conv = autocorr(avg_plaquettes_history_conv[:, 1])

avg_plaquettes_autocorr_conv = (avg_plaquettes_autocorr0_conv
                                + avg_plaquettes_autocorr1_conv) / 2

In [ ]:
steps_conv = np.arange(len(top_charges_autocorr_conv))

fig, ax = plt.subplots()
#ax.plot(steps_hmc, top_charges_autocorr_hmc, 
#        marker='', ls='-', label='topological_charge (HMC)')
#ax.plot(steps_conv, avg_plaquettes_autocorr_conv,
#        marker='', ls='--', label='avg plaquettes (L2HMC, ConvNet)')
ax.plot(steps_conv, top_charges_autocorr_conv,
        marker='', ls='-', label='topological charges (L2HMC, ConvNet)')
#ax.plot(steps1, top_charges_autocorr, marker='', ls='-', label='topological_charge (L2HMC)')
ax.set_ylabel('Autocorrelation', fontsize=14)
ax.set_xlabel('Gradient Evaluations', fontsize=14)
ax.set_title("")
ax.legend(loc='best', fontsize=12)
fig.savefig(os.path.join(model_conv.figs_dir, 
                         'top_charge_autocorrelation_fn_hmc.pdf'), 
            dpi=400, bbox_inches='tight')
#ax.set_xlim((ax.get_xlim()[0], 1000))
#ax.set_xlim((-5, 1000))
plt.show()

#### Using HMC model: `model_hmc`

In [ ]:
samples_hmc = np.array(
    model_hmc.lattice.samples.reshape((model_hmc.batch_size, -1)),
    dtype=np.float32
)
apply_transition_hmc = tfe.defun(model_hmc.dynamics.apply_transition)

In [ ]:
samples_hmc = tf.random_normal(shape=model_hmc.samples.shape)
samples_history_hmc = []
actions_history_hmc = []
avg_plaquettes_history_hmc = []
top_charges_history_hmc = []

In [ ]:
for i in range(5000):
    samples_history_hmc.append(samples_hmc.numpy())
    t0 = time.time()
    _, _, _, samples_hmc = apply_transition_hmc(samples_hmc)
    observables_hmc = np.array(
        model_hmc.lattice.calc_plaq_observables(samples_hmc)
    ).T
    actions_history_hmc.append(observables_hmc[0])
    avg_plaquettes_history_hmc.append(observables_hmc[1])
    top_charges_history_hmc.append(observables_hmc[2])
    print(f'step: {i}  time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_hmc = np.array(samples_history_hmc)
actions_history_hmc = np.array(actions_history_hmc)
avg_plaquettes_history_hmc = np.array(avg_plaquettes_history_hmc)
top_charges_history_hmc = np.array(top_charges_history_hmc)

In [ ]:
top_charges_autocorr0_hmc = autocorr(top_charges_history_hmc[:, 0])
top_charges_autocorr1_hmc = autocorr(top_charges_history_hmc[:, 1])

top_charges_autocorr_hmc = (top_charges_autocorr0_hmc 
                            + top_charges_autocorr1_hmc) / 2

##### Compute Autocorrelation spectrum directly from samples

In [ ]:
samples_ac_spectrum_hmc = compute_ac_spectrum(samples_history_hmc, 
                                              samples_history_mean_hmc)

steps_hmc = np.arange(len(samples_ac_spectrum_hmc))

In [ ]:
samples_ac_spectrum_hmc_file = os.path.join(model_hmc.info_dir,
                                            'samples_ac_spectrum_hmc.pkl')
top_charges_acl_spectrum_hmc_file = os.path.join(model_hmc.info_dir,
                                                 'top_charges_acl_spectrum_hmc.pkl')
actions_acl_spectrum_hmc_file = os.path.join(model_hmc.info_dir,
                                             'actions_acl_spectrum_hmc.pkl')
avg_plaquette_acl_spectrum_hmc_file = os.path.join(model_hmc.info_dir,
                                                   'avg_plaquette_acl_spectrum_hmc.pkl')
with open(samples_ac_spectrum_hmc_file, 'wb') as f:
    pickle.dump(samples_ac_spectrum_hmc, f)

with open(top_charges_acl_spectrum_hmc_file, 'wb') as f:
    pickle.dump(top_charges_acl_spectrum_hmc, f)
    
with open(actions_acl_spectrum_hmc_file, 'wb') as f:
    pickle.dump(actions_acl_spectrum_hmc, f)
    
with open(avg_plaquette_acl_spectrum_hmc_file, 'wb') as f:
    pickle.dump(avg_plaquette_acl_spectrum_hmc, f)

In [ ]:
steps = np.arange(len(samples_ac_spectrum))

In [ ]:
fig, ax = plt.subplots()
ax.plot(steps_hmc, samples_ac_spectrum_hmc, marker='', ls='-', label='HMC')
ax.plot(steps, samples_ac_spectrum, marker='', ls='-', label='L2HMC')
ax.set_ylabel('Autocorrelation', fontsize=14)
ax.set_xlabel('Gradient Evaluations', fontsize=14)
ax.set_title("Autocorrelation directly from samples (Eq. 15)", fontsize=16)
ax.legend(loc='best')
fig.savefig(os.path.join(model_hmc.figs_dir, 'samples_autocorrelation_fn_hmc.pdf'), 
            dpi=400, bbox_inches='tight')
plt.show()

##### Compute Autocorrelation for Topological charge

In [ ]:
#top_charges_acl_spectrum_hmc = compute_autocorrelation(top_charges_history_hmc, 
#                                                       top_charges_mean_hmc,
#                                                       top_charges_var_hmc)
#actions_acl_spectrum_hmc = compute_autocorrelation(actions_history_hmc, 
#                                                   actions_mean_hmc,
#                                                   actions_var_hmc)
#avg_plaquette_acl_spectrum_hmc = compute_autocorrelation(avg_plaquettes_history_hmc,
#                                                         avg_plaquettes_mean_hmc,
#                                                         avg_plaquettes_var_hmc)
#steps_hmc = np.arange(len(top_charges_acl_spectrum_hmc))

In [ ]:
#top_charges_acl_spectrum_hmc = compute_autocorrelation(top_charges_history_hmc, 
#                                                       top_charges_mean_hmc,
#                                                       top_charges_var_hmc)

In [ ]:
top_charges_autocorr0 = autocorr(top_charges_history[:, 0])
top_charges_autocorr1 = autocorr(top_charges_history[:, 1])

In [ ]:
#top_charges_autocorr = (top_charges_autocorr0 + top_charges_autocorr1) / 2
#avg_plaquette_autocorr0 = autocorr(avg_plaquettes_history[:, 0])
#avg_plaquette_autocorr1 = autocorr(avg_plaquettes_history[:, 1])

In [ ]:
avg_plaquette_autocorr0_hmc = autocorr(avg_plaquettes_history_hmc[:, 0])
avg_plaquette_autocorr1_hmc = autocorr(avg_plaquettes_history_hmc[:, 1])

#avg_plaquette_autocorr = (avg_plaquette_autocorr0
#                          + avg_plaquette_autocorr1) / 2

avg_plaquette_autocorr_hmc = (avg_plaquette_autocorr0_hmc
                              + avg_plaquette_autocorr1_hmc) / 2

#steps = np.arange(len(avg_plaquette_autocorr))
steps_hmc = np.arange(len(avg_plaquette_autocorr_hmc))

In [ ]:
fig, ax = plt.subplots()
ax.plot(steps_hmc, avg_plaquette_autocorr_hmc, marker='', ls='-', label='Average plaquette (HMC)')
ax.plot(steps_conv, avg_plaquette_autocorr_conv, marker='', ls='-', label='Average plaquette (L2HMC, ConvNet)')
#ax.plot(steps, avg_plaquette_autocorr, marker='', ls='-', label='Average plaquette (L2HMC)')
#ax.plot(steps, avg_plaquette_autocorr1, marker='', ls='-', label='Average plaquette1 (L2HMC)')
ax.set_ylabel('Autocorrelation', fontsize=14)
ax.set_xlabel('Gradient Evaluations', fontsize=14)
ax.legend(loc='best', fontsize=12)
#fig.savefig(os.path.join(model_hmc.figs_dir, 'avg_plaquette_autocorrelation_fn_hmc.pdf'), 
#            dpi=400, bbox_inches='tight')
#ax.set_xlim((-20, 450))
plt.show()

In [ ]:
%matplotlib notebook

In [ ]:
steps1 = np.arange(len(top_charges_autocorr0))
steps_hmc = np.arange(len(top_charges_autocorr0_hmc))
steps_conv = np.arange(len(top_charges_autocorr_conv))

fig, ax = plt.subplots()
ax.plot(steps_hmc, top_charges_autocorr_hmc, 
        marker='', ls='-', label='topological_charge (HMC)')
ax.plot(steps_conv, top_charges_autocorr_conv,
        marker='', ls='-', label='topological_charges (L2HMC, ConvNet)')
#ax.plot(steps1, top_charges_autocorr, marker='', ls='-', label='topological_charge (L2HMC)')
ax.set_ylabel('Autocorrelation', fontsize=14)
ax.set_xlabel('Gradient Evaluations', fontsize=14)
ax.set_title("")
ax.legend(loc='best', fontsize=12)
fig.savefig(os.path.join(model_hmc.figs_dir, 
                         'top_charge_autocorrelation_fn_hmc.pdf'), 
            dpi=400, bbox_inches='tight')
#ax.set_xlim((-20, 500))
plt.show()

In [ ]:
fig, ax = plt.subplots()
#ax.semilogy(steps_hmc, actions_acl_spectrum_hmc, marker='', ls='-', label='total action (HMC)')
ax.semilogy(steps_hmc, avg_plaquette_acl_spectrum_hmc, marker='', ls='-', label='average plaquette (HMC)')
#ax.plot(steps, actions_acl_spectrum, marker='', ls='-', label='total action (L2HMC)')
ax.semilogy(steps, avg_plaquette_acl_spectrum, marker='', ls='-', label='average plaquette (L2HMC)')
ax.set_ylabel('Autocorrelation', fontsize=14)
ax.set_xlabel('Gradient Evaluations', fontsize=14)
ax.set_title("")
ax.legend(loc='best', fontsize=12)
fig.savefig(os.path.join(model_hmc.figs_dir, 'avg_plaquette_autocorrelation_fn_hmc_semilogy.pdf'), 
            dpi=400, bbox_inches='tight')
plt.show()

#### Using GENERIC NET (Fully-Connected) model: `model_fc`

In [ ]:
apply_transition_fc = tfe.defun(model_fc.dynamics.apply_transition)

In [ ]:
  #samples_history = []
  #for i in range(eval_iters):
    #samples_history.append(samples.numpy())
    #_, _, _, samples = apply_transition(samples)
  #samples_history = np.array(samples_history)
  #print("Sampling complete.")
  #sys.stdout.flush()

In [ ]:
#samples_fc = tf.random_normal(shape=model_fc.samples.shape)
samples_fc = model_fc.samples
samples_history_fc = []
actions_history_fc = []
avg_plaquettes_history_fc = []
top_charges_history_fc = []

In [ ]:
_, _, _, _samples_fc = apply_transition_fc(samples_fc)

In [ ]:
_observables_fc = model_fc.lattice.calc_plaq_observables(_samples_fc.numpy())

In [ ]:
_observables_fc

In [ ]:
for i in range(1000):
    samples_history_fc.append(samples_fc.numpy())
    t0 = time.time()
    _, _, _, samples_fc = apply_transition_fc(samples_fc)
    observables_fc = np.array(
        model_fc.lattice.calc_plaq_observables(samples_fc.numpy())
    ).T
    actions_history_fc.append(observables_fc[0])
    avg_plaquettes_history_fc.append(observables_fc[1])
    top_charges_history_fc.append(observables_fc[2])
    step_time = (time.time() - t0) / (model_fc.num_steps * model_fc.batch_size)
    print(f'step: {i}  time/step: {step_time:^6.4g} '
          f'top_charge: {np.mean(observables_conv[2]):^6.4g} '
          f'avg_plaq: {np.mean(observables_conv[1]):^6.4g}')
    #print(f'step: {i}  time/step: {step_time:^6.4g}  avg_plaq: {np.mean(observables_fc[1]):^6.4g}')

In [ ]:
samples_history_fc = np.array(samples_history_fc)
actions_history_fc = np.array(actions_history_fc)
avg_plaquettes_history_fc = np.array(avg_plaquettes_history_fc)
top_charges_history_fc = np.array(top_charges_history_fc)

In [ ]:
top_charges_autocorr0_fc = autocorr(top_charges_history_fc[:, 0])
top_charges_autocorr1_fc = autocorr(top_charges_history_fc[:, 1])

top_charges_autocorr_fc = (top_charges_autocorr0_fc 
                            + top_charges_autocorr1_fc) / 2

In [ ]:
avg_plaquettes_autocorr0_fc = autocorr(avg_plaquettes_history_fc[:, 0])
avg_plaquettes_autocorr1_fc = autocorr(avg_plaquettes_history_fc[:, 1])

avg_plaquettes_autocorr_fc = (avg_plaquettes_autocorr0_fc
                              + avg_plaquettes_autocorr1_fc) / 2

###### Topological Charge Autocorrelation

In [ ]:
steps_fc = np.arange(len(top_charges_autocorr_fc))

fig, ax = plt.subplots()
#ax.plot(steps_hmc, top_charges_autocorr_hmc, 
#        marker='', ls='-', label='topological_charge (HMC)')
ax.plot(steps_fc, top_charges_autocorr_fc,
        marker='', ls='-', label='topological_charges (L2HMC, FCNet)')
#ax.plot(steps1, top_charges_autocorr, marker='', ls='-', label='topological_charge (L2HMC)')
ax.set_ylabel('Autocorrelation', fontsize=14)
ax.set_xlabel('Gradient Evaluations', fontsize=14)
ax.set_title("")
ax.legend(loc='best', fontsize=12)
fig.savefig(os.path.join(model_fc.figs_dir, 
                         'top_charge_autocorrelation_fn_hmc.pdf'), 
            dpi=400, bbox_inches='tight')
#ax.set_xlim((-20, 500))
plt.show()

###### Average Plaquette Autocorrelation

In [ ]:
steps_fc = np.arange(len(avg_plaquettes_autocorr_fc))

fig, ax = plt.subplots()
#ax.plot(steps_hmc, avg_plaquettes_autocorr_hmc, 
#        marker='', ls='-', label='topological_charge (HMC)')
ax.plot(steps_fc, avg_plaquettes_autocorr_fc,
        marker='', ls='-', label='Average Plaquettes (L2HMC, FCNet)')
#ax.plot(steps1, avg_plaquettes_autocorr, marker='', ls='-', label='topological_charge (L2HMC)')
ax.set_ylabel('Autocorrelation', fontsize=14)
ax.set_xlabel('Gradient Evaluations', fontsize=14)
ax.set_title("")
ax.legend(loc='best', fontsize=12)
fig.savefig(os.path.join(model_fc.figs_dir, 
                         'avg_plaquettes_autocorrelation_fn_hmc.pdf'), 
            dpi=400, bbox_inches='tight')
#ax.set_xlim((-20, 500))
plt.show()

In [ ]:
apply_transition_conv = tfe.defun(model_conv.dynamics.apply_transition)

In [ ]:
samples_fc = np.array(model.lattice.samples.reshape((model.batch_size, -1)), dtype=np.float32)
print(samples.shape)

In [ ]:
apply_transition = tfe.defun(model.dynamics.apply_transition)

In [ ]:
samples = tf.random_normal(shape=model.samples.shape)
samples_history = []
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for i in range(5000):
    samples_history.append(samples.numpy())
    t0 = time.time()
    _, _, _, samples = apply_transition(samples)
    observables = np.array(model.lattice.calc_plaq_observables(samples)).T
    actions_history.append(observables[0])
    avg_plaquettes_history.append(observables[1])
    top_charges_history.append(observables[2])
    print(f'step: {i}  time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history = np.array(samples_history)
actions_history = np.array(actions_history)
avg_plaquettes_history = np.array(avg_plaquettes_history)
top_charges_history = np.array(top_charges_history)

In [ ]:
top_charges_autocorr0_conv = autocorr(top_charges_history_conv[:, 0])
top_charges_autocorr1_conv = autocorr(top_charges_history_conv[:, 1])
avg_plaquette_autocorr0_conv = autocorr(avg_plaquettes_history_conv[:, 0])
avg_plaquette_autocorr1_conv = autocorr(avg_plaquettes_history_conv[:, 1])

top_charges_autocorr_conv = (top_charges_autocorr0_conv 
                             + top_charges_autocorr1_conv) / 2
avg_plaquette_autocorr_conv = (avg_plaquette_autocorr0_conv
                               + avg_plaquette_autocorr1_conv) / 2

##### Compute Autocorrelation spectrum directly from samples

In [ ]:
samples_ac_spectrum = compute_ac_spectrum(samples_history, 
                                          samples_history_mean,
                                          samples_history_var)

#ax.plot(steps, samples_ac_spectrum, marker='', ls='-', label='L2HMC')
steps = np.arange(len(samples_ac_spectrum))

In [ ]:
%matplotlib notebook

In [ ]:
fig, ax = plt.subplots()
ax.plot(steps, samples_ac_spectrum, marker='', ls='-', label='L2HMC')
ax.set_ylabel('Autocorrelation', fontsize=14)
ax.set_xlabel('Gradient Evaluations', fontsize=14)
ax.set_title("Autocorrelation directly from samples (Eq. 15)", fontsize=16)
ax.legend(loc='best')
fig.savefig(os.path.join(model.figs_dir, 'samples_autocorrelation_fn.pdf'), 
            dpi=400, bbox_inches='tight')
plt.show()

##### Compute Autocorrelation for Topological charge

In [ ]:
top_charges_acl_spectrum = compute_autocorrelation(top_charges_history, 
                                                   top_charges_mean,
                                                   top_charges_var)
actions_acl_spectrum = compute_autocorrelation(actions_history, 
                                               actions_mean,
                                               actions_var)
avg_plaquette_acl_spectrum = compute_autocorrelation(avg_plaquettes_history,
                                                     avg_plaquettes_mean,
                                                     avg_plaquettes_var)
steps = np.arange(len(top_charges_acl_spectrum))

In [ ]:
fig, ax = plt.subplots()
ax.plot(steps, top_charges_acl_spectrum, marker='', ls='-', label='topological_charge (L2HMC)')
ax.plot(steps, actions_acl_spectrum, marker='', ls='-', label='total action (L2HMC)')
ax.plot(steps, avg_plaquette_acl_spectrum, marker='', ls='-', label='average plaquette (L2HMC)')
ax.set_ylabel('Autocorrelation', fontsize=14)
ax.set_xlabel('Gradient Evaluations', fontsize=14)
ax.set_title("")
ax.legend(loc='best', fontsize=12)
fig.savefig(os.path.join(model.figs_dir, 'observables_autocorrelation_fn.pdf'), 
            dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
samples_ac_spectrum_file = os.path.join(model.info_dir,
                                        'samples_ac_spectrum.pkl')
top_charges_acl_spectrum_file = os.path.join(model.info_dir,
                                             'top_charges_acl_spectrum.pkl')
actions_acl_spectrum_file = os.path.join(model.info_dir,
                                         'actions_acl_spectrum.pkl')
avg_plaquette_acl_spectrum_file = os.path.join(model.info_dir,
                                               'avg_plaquette_acl_spectrum.pkl')
with open(samples_ac_spectrum_file, 'wb') as f:
    pickle.dump(samples_ac_spectrum, f)

with open(top_charges_acl_spectrum_file, 'wb') as f:
    pickle.dump(top_charges_acl_spectrum, f)
    
with open(actions_acl_spectrum_file, 'wb') as f:
    pickle.dump(actions_acl_spectrum, f)
    
with open(avg_plaquette_acl_spectrum_file, 'wb') as f:
    pickle.dump(avg_plaquette_acl_spectrum, f)

### Load / Restore from previous run

In [ ]:
model_l2hmc = GaugeModelEager(params=params, log_dir='../../gauge_logs_eager/run_40/', restore=True, use_defun=True)

In [ ]:
avg_plaq_arr_hmc = model.data['average_plaquettes_arr']
top_charge_arr_hmc = model.data['topological_charges_arr']
total_actions_arr_hmc = model.data['total_actions_arr']
steps = np.arange(500)

In [ ]:
avg_plaq_arr_hmc.shape

In [ ]:
avg_plaq_arr_l2hmc

In [ ]:
steps = np.arange(500)

#### Compare plots across runs

In [ ]:
plot_run_data(model.data, model.params, steps, model.figs_dir, skip_steps=1)

In [ ]:
average_plaquettes_file_l2hmc = os.path.join(model_l2hmc.info_dir,
                                             'average_plaquettes.npy')
top_charge_file_l2hmc = os.path.join(model_l2hmc.info_dir,
                                     'topological_charges.npy')
total_actions_file_l2hmc = os.path.join(model_l2hmc.info_dir,
                                        'total_actions.npy')
avg_plaq_arr_l2hmc = np.load(average_plaquettes_file_l2hmc)
top_charge_arr_l2hmc = np.load(top_charge_file_l2hmc)
total_actions_arr_l2hmc = np.load(total_actions_file_l2hmc)

In [ ]:
model_l2hmc.topological_charges_arr = []
model_l2hmc.average_plaquettes_arr = []
model_l2hmc.total_actions_arr = []

In [ ]:
model_l2hmc._update_data(total_actions_arr_l2hmc,
                         avg_plaq_arr_l2hmc,
                         top_charge_arr_l2hmc)

In [ ]:
fig, ax = plt.subplots()
ax.plot(steps, avg_plaq_arr_hmc[:, 0], marker='', markersize=4., ls='--', label=f'Generic HMC (sample 0) (avg: {np.mean(avg_plaq_arr_hmc[:, 0]):^5.4g})')
ax.plot(steps, avg_plaq_arr_hmc[:, 1], marker='', markersize=4., ls='--', label=f'Generic HMC (sample 1) (avg: {np.mean(avg_plaq_arr_hmc[:, 1]):^5.4g})')
ax.plot(steps, avg_plaq_arr_l2hmc[:, 0], marker='', markersize=4., ls='-', label=f'L2HMC (sample 0) (avg: {np.mean(avg_plaq_arr_l2hmc[:, 0]):^5.4g})')
ax.plot(steps, avg_plaq_arr_l2hmc[:, 1], marker='', markersize=4., ls='-', label=f'L2HMC (sample 1) (avg: {np.mean(avg_plaq_arr_l2hmc[:, 1]):^5.4g})')
#ax.plot(steps, avg_plaq_arr_l2hmc[:, 0], marker='.', ls='-', label=f'L2HMC (avg: {np.mean(avg_plaq_arr[:, 1]):^5.4g})')
#ax.axhline(np.mean(avg_plaq_arr[:,0]), xmin=0, xmax=500, color='C0', ls='-', lw=2., label='Sample 1 (avg)')
#ax.axhline(np.mean(avg_plaq_arr[:,1]), xmin=0, xmax=500, color='C1', ls='-', lw=2., label='Sample 2 (avg)')
ax.axhline(u1_plaq_exact(params['beta']), xmin=0, xmax=500, color='k', lw=2., label=f"Exact ({u1_plaq_exact(params['beta']):^5.4g})")
ax.set_ylabel('Avg. Plaquette')
ax.set_xlabel('Step')
ax.set_xlim((0, 20))
ax.legend(loc='best')
#file_name = os.path.join(model.figs_dir, 'average_plaquettes.pdf')
#fig.savefig(file_name, dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots()
#ax.plot(steps[::5], top_charge_arr[:, 0][::5], marker='.', ls='-', label='sample 1')
#ax.plot(steps[::5], top_charge_arr[:, 1][::5], marker='.', ls='-', label='sample 2')
_ = ax.plot(steps, top_charge_arr_hmc[:, 0], marker='', markersize=4., ls='--', label=f'Generic HMC (sample 0) (avg: {np.mean(top_charge_arr_hmc[:, 0]):^5.4g})')
_ = ax.plot(steps, top_charge_arr_hmc[:, 1], marker='', markersize=4., ls='--', label=f'Generic HMC (sample 1) (avg: {np.mean(top_charge_arr_hmc[:, 1]):^5.4g})')
_ = ax.plot(steps, top_charge_arr_l2hmc[:, 0], marker='', markersize=4., ls='-', label=f'L2HMC (sample 0) (avg: {np.mean(top_charge_arr_l2hmc[:, 0]):^5.4g})')
_ = ax.plot(steps, top_charge_arr_l2hmc[:, 1], marker='', markersize=4., ls='-', label=f'L2HMC (sample 1) (avg: {np.mean(top_charge_arr_l2hmc[:, 1]):^5.4g})')
#ax.axhline(u1_plaq_exact(params['beta']), xmin=0, xmax=500, color='r', label='Exact')
_ = ax.set_ylabel('Topological Charge')
_ = ax.set_xlabel('Step')
_ = ax.legend(loc='best')
_ = ax.set_xlim((0, 25))
_ = plt.show()

In [ ]:
fig, ax = plt.subplots()
#ax.plot(steps[::5], total_action_arr[:, 0][::5], marker='.', ls='-', label='sample 1')
#ax.plot(steps[::5], total_action_arr[:, 1][::5], marker='.', ls='-', label='sample 2')
_ = ax.plot(steps, total_action_arr_hmc[:, 0], marker='', markersize=4., ls='--', label=f'Generic HMC (sample 0) (avg: {np.mean(total_action_arr_hmc[:, 0]):^5.4g})')
_ = ax.plot(steps, total_action_arr_hmc[:, 1], marker='', markersize=4., ls='--', label=f'Generic HMC (sample 1) (avg: {np.mean(total_action_arr_hmc[:, 1]):^5.4g})')
_ = ax.plot(steps, total_actions_arr_l2hmc[:, 0], marker='', markersize=4., ls='-', label=f'L2HMC (sample 0) (avg: {np.mean(total_actions_arr_l2hmc[:, 0]):^5.4g})')
_ = ax.plot(steps, total_actions_arr_l2hmc[:, 1], marker='', markersize=4., ls='-', label=f'L2HMC (sample 1) (avg: {np.mean(total_actions_arr_l2hmc[:, 1]):^5.4g})')
#ax.axhline(u1_plaq_exact(params['beta']), xmin=0, xmax=500, color='r', label='Exact')
_ = ax.set_ylabel('Total Action')
_ = ax.set_xlabel('Step')
_ = ax.legend(loc='best')
_ = ax.set_xlim((0, 25))
_ = plt.show()

In [ ]:
avg_plaq1_mean, avg_plaq1_err = calc_avg_vals_errors(avg_plaq_arr[100:, 0], num_blocks=100)
avg_plaq2_mean, avg_plaq2_err = calc_avg_vals_errors(avg_plaq_arr[100:, 1], num_blocks=100)
print(avg_plaq1_mean, avg_plaq1_err)
print(avg_plaq2_mean, avg_plaq2_err)

In [ ]:
u1_plaq_exact(params['beta'])

In [ ]:
print(model.learning_rate.eval())

In [ ]:
model.data['avg_top_charge']

In [ ]:
avg_plaqs = np.array(model.average_plaquettes_arr)

In [ ]:
avg_plaqs.mean()

In [ ]:
u1_plaq_exact(params['beta'])

## Build tf.keras.Model using tf.data.Dataset object

In [ ]:
from u1_model_eager import GaugeModelEager, train_one_iter

In [ ]:
from lattice.gauge_lattice import GaugeLattice, u1_plaq_exact
import l2hmc_eager.gauge_dynamics_eager as gde

In [ ]:
lattice = GaugeLattice(time_size=8, 
                       space_size=8, 
                       dim=2, 
                       beta=8., 
                       link_type='U1',
                       num_samples=2,
                       rand=False)

In [ ]:
lattice.calc_plaq_observables(lattice.samples)

In [ ]:
lattice.total_action(lattice.samples)

In [ ]:
potential_fn = lattice.get_energy_function(lattice.samples)

In [ ]:
dynamics = gde.GaugeDynamicsEager(lattice=lattice,
                                  num_steps=10,
                                  eps=0.1,
                                  minus_loglikelihood_fn=potential_fn,
                                  conv_net=True,
                                  hmc=False)

In [ ]:
samples_tensor = tf.convert_to_tensor(lattice.samples, dtype=tf.float32)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(samples_tensor)

In [ ]:
dataset.output_shapes, dataset.output_types

In [ ]:
dataset.

In [ ]:
dataset = dataset.map(dynamics.apply_transition)

## Iterate over different `eps` and `n_steps`

In [ ]:
eps_arr = np.linspace(0.02, 0.2, 10)
n_steps_arr = [5, 10, 15]

for eps in eps_arr:
    for n_steps in n_steps_arr:
        params['eps'] = eps
        params['n_steps'] = n_steps
        
        tf.reset_default_graph()
        model = GaugeModelEager(params=params,
                                log_dir=None,
                                restore=False,
                                use_defun=True)
        _ = model.calc_observables(model.samples, _print=True, _write=True)
        model.train(1)
        
        model.train(1000)

In [ ]:
tf.reset_default_graph()

In [ ]:
avg_plaq_arr = np.array(model.average_plaquettes_arr)

In [ ]:
avg_plaq_arr.shape

In [ ]:
avg_plaq_arr.mean(axis=0)

In [ ]:
u1_plaq_exact(params['beta'])

In [ ]:
%debug

# OLD

## Using Graph mode

In [ ]:
params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 3.5,
    'num_samples': 2,
    'n_steps': 5,
    'eps': 0.05,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-4,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.96,
    'train_steps': 1000,
    'record_loss_every': 50,
    'data_steps': 10,
    'save_steps': 50,
    'print_steps': 1,
    'logging_steps': 5,
    'clip_value': 100,
    'rand': False,
    'metric': 'l2',
    'conv_net': True,
    'hmc': False
}

In [ ]:
def graph_step(dynamics, optimizer, samples, params, global_step=None):
    clip_val = params.get('clip_value', 10)
    loss, samples_out, accept_prob, grads = gde.loss_and_grads(
        dynamics=dynamics, 
        x=samples, 
        params=params,
        loss_fn=gde.compute_loss,
        defun=False
    )
    gradients, _ = tf.clip_by_global_norm(grads, clip_val)
    train_op = optimizer.apply_gradients(zip(gradients, dynamics.variables))
    return train_op, loss, samples_out

In [ ]:
with tf.Graph().as_default():
    lattice = GaugeLattice(time_size=params.get('time_size', 8),
                           space_size=params.get('space_size', 8),
                           dim=params.get('dim', 2),
                           beta=params.get('beta', '2.5'),
                           link_type=params.get('link_type', 'U1'),
                           num_samples=params.get('num_samples', 2),
                           rand=params.get('rand', False))
    batch_size = lattice.samples.shape[0]
    samples = tf.convert_to_tensor(
        lattice.samples.reshape((batch_size, -1)),
        dtype=tf.float32
    )
    potential_fn = lattice.get_energy_function(samples)

    dynamics =  gde.GaugeDynamicsEager(lattice=lattice,
                                       n_steps=params.get('n_steps', 10),
                                       eps=params.get('eps', 0.1),
                                       minus_loglikelihood_fn=potential_fn,
                                       conv_net=params.get('conv_net', True),
                                       hmc=params.get('hmc', False),
                                       defun=False)
    x = tf.placeholder(tf.float32, shape=samples.shape)
    x_, v_, x_accept_prob, x_out = dynamics.apply_transition(x)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        np_x_, np_v_, np_x_accept_prob, np_x_out =  sess.run(
            [x_, v_, x_accept_prob, x_out],
            feed_dict={x: lattice.samples.reshape((batch_size, -1))}
        )

In [ ]:
#tf.reset_default_graph()
#with tf.Graph().as_default():
lattice = GaugeLattice(time_size=params.get('time_size', 8),
                       space_size=params.get('space_size', 8),
                       dim=params.get('dim', 2),
                       beta=params.get('beta', '2.5'),
                       link_type=params.get('link_type', 'U1'),
                       num_samples=params.get('num_samples', 2),
                       rand=params.get('rand', False))
batch_size = lattice.samples.shape[0]
samples_tensor = tf.convert_to_tensor(
    lattice.samples.reshape((batch_size, -1)),
    dtype=tf.float32
)
potential_fn = lattice.get_energy_function(samples_tensor)

dynamics =  gde.GaugeDynamicsEager(lattice=lattice,
                                   n_steps=params.get('n_steps', 10),
                                   eps=params.get('eps', 0.1),
                                   minus_loglikelihood_fn=potential_fn,
                                   conv_net=params.get('conv_net', True),
                                   hmc=params.get('hmc', False),
                                   defun=False)

samples = lattice.samples.reshape((batch_size, -1))
x = tf.placeholder(tf.float32, shape=samples_tensor.shape)
loss, _, _ = gde.compute_loss(dynamics, x, params, defun=False)

optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate_init'])
train_op, loss, x_out = graph_step(dynamics, optimizer, x, params)

session_conf = tf.ConfigProto()

In [ ]:
#with tf.Session(config=session_conf) as sess:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# Warmup to reduce initialization effect when timing
for _ in range(1):
    _, _ = sess.run([train_op, loss], feed_dict={x: samples})

# Training
#start_time = time.time()
for i in range(100):
    t0 = time.time()
    _, loss_np, samples = sess.run([train_op, loss, x_out],
                                       feed_dict={x: samples})
    print(f"step: {i}  loss: {loss_np:^6.4g}  time/step: {time.time() - t0:^6.4g}")

## Helper Functions

In [ ]:
def train_one_iter(dynamics, x, optimizer, loss_fn=l2hmc.compute_loss, 
                   scale=0.1, eps=1e-4, metric='l2', clip_value=10, 
                   global_step=None):
    loss, grads, out, accept_prob = l2hmc.loss_and_grads(
        dynamics, x, loss_fn=loss_fn, scale=scale, eps=eps, metric=metric
    )
    gradients, _ = tf.clip_by_global_norm(grads, clip_value)
    optimizer.apply_gradients(
        zip(grads, dynamics.trainable_variables), global_step=global_step
    )
    return loss, out, accept_prob, gradients

In [ ]:
def write_run_data(file_path, data, write_mode='a'):
    with open(file_path, write_mode) as f:
        f.write('\n')
        info_str = (f"step: {data['step']:<3g} loss: {data['loss']:^6.5g} "
                    f" time/step: {data['time']:^6.5g} "
                    f" accept: {data['accept_prob']:^6.5g} "
                    f" eps: {data['eps']:^6.5g} "
                    f" avg_S: {data['avg_S']:^6.5g} "
                    f" avg_topQ: {data['avg_top_charge']:^6.5g} "
                    f" avg_plaq: {data['avg_plaq']:^6.5g} \n")
        f.write(info_str)
        f.write('\n')
        f.write('avg_plaquettes: {}\n'.format(data['avg_plaquettes']))
        f.write('topological_charges: {}\n'.format(data['top_charges']))
        f.write('total_actions: {}\n'.format(data['total_actions']))
        f.write((len(info_str) + 1)*'-')

In [ ]:
def write_run_parameters(file_path, parameters):
    with open(file_path, 'w') as f:
        f.write('Parameters:\n')
        f.write(80 * '-' + '\n')
        for key, val in parameters.items():
            f.write(f'{key}: {val}\n')
        f.write(80*'=')
        f.write('\n')

In [ ]:
def print_run_data(data):
    print(f"\nstep: {data['step']:<3g} loss: {data['loss']:^6.5g} "
          f" time/step: {data['time']:^6.5g} "
          f" accept: {data['accept_prob']:^6.5g} "
          f" eps: {data['eps']:^6.5g} "
          f" avg_S: {data['avg_S']:^6.5g} "
          f" avg_topQ: {data['avg_top_charge']:^6.5g} "
          f" avg_plaq: {data['avg_plaq']:^6.5g} \n")
    print('avg_plaquettes: {}\n'.format(data['avg_plaquettes']))

In [ ]:
def save_run_data(checkpointer, log_dir, files, data):
    saved_path = checkpointer.save(file_prefix=os.path.join(log_dir,
                                                            "ckpt"))
    print(f"Saved checkpoint to: {saved_path}")
    np.save(files['avg_plaquettes_file'], data['avg_plaquettes_arr'])
    np.save(files['total_actions_file'], data['total_actions_arr'])
    np.save(files['top_charges_file'], data['top_charges_arr'])
    np.save(files['samples_file'], data['samples'])

In [ ]:
def write_summaries(summary_writer, data):
    with summary_writer.as_default():
        with tf.contrib.summary.always_record_summaries():
            tf.contrib.summary.scalar("Training_loss", data['loss'],
                                      step=data['global_step'])
            tf.contrib.summary.scalar("avg_plaquettes", data['avg_plaq'],
                                      step=data['global_step'])
            tf.contrib.summary.scalar("total_actions", data['avg_S'],
                                      step=data['global_step'])
            tf.contrib.summary.scalar("top_charges", data['avg_top_charge'],
                                      step=data['global_step'])

In [ ]:
def exact_plaquette_average(beta):
    return i1(beta) / i0(beta)

## Construct GaugeLattice with $U(1)$ gauge group

$$ S[\theta] = \beta \sum_{x;\,\, \nu\neq\mu} 1 - \cos(\theta_{\mu\nu})$$
with $\theta_{\mu\nu} \equiv \theta_{\mu}(x) + \theta_{\nu}(x +\hat \mu) - \theta_{\mu}(x + \hat \nu) - \theta_{\nu}(x)$ 

### Run L2HMC for $U(1)$ gauge model

In [ ]:
##########################  Parameters  #####################################
# n_steps: number of leapfrog steps, eps: initial step size for dynamics
# loss_scale: scaling factor (lambda^2 in paper) in loss objective
# loss_eps: for numeric stability in loss function
# beta: inverse coupling strength
##############################################################################
params = {
    'time_size': 8,
    'space_size': 8,
    'dim': 2,
    'beta': 3.5,
    'num_samples': 2,
    'n_steps': 10,
    'eps': 0.05,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate': 1e-4,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.96,
    'train_iters': 500,
    'record_loss_every': 50,
    'save_steps': 50,
    'clip_value': 100,
    'rand': False,
    'conv_net': True,
    'metric': 'l2'
}

In [ ]:
u1_lattice = GaugeLattice(time_size=params['time_size'], 
                          space_size=params['space_size'], 
                          dim=params['dim'], 
                          beta=params['beta'],
                          link_type='U1', 
                          num_samples=params['num_samples'], 
                          rand=params['rand'])
if params['conv_net']:
    u1_samples_tensor = tf.convert_to_tensor(u1_lattice.samples, 
                                             dtype=tf.float32)
else:
    flat_samples = [sample.flatten() for sample in u1_lattice.samples]
    u1_samples_tensor = tf.convert_to_tensor(np.stack(flat_samples), 
                                             dtype=tf.float32)

# Construct dynamics object
u1_energy_fn = u1_lattice.get_energy_function(u1_samples_tensor)
u1_dynamics = l2hmc.GaugeDynamicsEager(lattice=u1_lattice, 
                                       n_steps=params['n_steps'],
                                       eps=params['eps'],
                                       minus_loglikelihood_fn=u1_energy_fn, 
                                       conv_net=params['conv_net'], 
                                       hmc=False)
total_actions = []
average_plaquettes = []
topological_charges = []
samples = u1_samples_tensor

In [ ]:
# Create new log_dir with new run number
root_dir = '../../U1_logs/'
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

log_dirs = os.listdir(root_dir)
try:
    run_nums = [int(i.split('_')[-1]) for i in log_dirs if i.startswith('run')]
    run_num = max(run_nums) + 1
except:
    run_num = 1
    
log_dir = f'../../U1_logs/run_{run_num}'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
print(f'{log_dir}')
    
run_files = {
    'parameters_file': os.path.join(log_dir, 'parameters.txt'),
    'run_info_file': os.path.join(log_dir, 'run_info.txt'),
    'avg_plaquettes_file':  os.path.join(log_dir, 'average_plaquettes.npy'),
    'total_actions_file': os.path.join(log_dir, 'total_actions.npy'),
    'top_charges_file': os.path.join(log_dir, 'topological_charges.npy'),
    'samples_file': os.path.join(log_dir, 'samples.npy'),
}

In [ ]:
global_step = tf.train.get_or_create_global_step()
_ = global_step.assign(1)

learning_rate = tf.train.exponential_decay(learning_rate=params['learning_rate'], 
                                           global_step=global_step, 
                                           decay_steps=params['learning_rate_decay_steps'],
                                           decay_rate=params['learning_rate_decay_rate'], 
                                           staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
checkpointer = tf.train.Checkpoint(
    optimizer=optimizer, dynamics=u1_dynamics, global_step=global_step
)
summary_writer = tf.contrib.summary.create_file_writer(log_dir)
loss_fn = l2hmc.compute_loss

print(u1_plaq_exact(params['beta']))

In [ ]:
################### SAVE INFO ABOUT INITIAL STATE OF LATTICE ###################
start_step = global_step.numpy()
observables = np.array(u1_lattice.calc_plaq_observables(samples))
_total_actions = observables[:, 0]
_avg_plaquettes = observables[:, 1]
_top_charges = observables[:, 2]

total_actions.append(_total_actions)
average_plaquettes.append(_avg_plaquettes)
topological_charges.append(_top_charges)

data = {
    'step': 0,
    'global_step': global_step.numpy(),
    'loss': 0.,
    'time': 0.,
    'accept_prob': 0.,
    'eps': u1_dynamics.eps.numpy(),
    'avg_S': np.mean(_total_actions),
    'avg_top_charge': np.mean(_top_charges),
    'avg_plaq': np.mean(_avg_plaquettes),
    'avg_plaquettes': _avg_plaquettes,
    'top_charges': _top_charges,
    'total_actions': _total_actions,
    'avg_plaquettes_arr': average_plaquettes,
    'top_charges_arr': topological_charges,
    'total_actions_arr': total_actions,
    'samples': samples.numpy()
}

_ = print_run_data(data)
_ = write_run_data(run_files['run_info_file'], data, 'w')
_ = write_run_parameters(run_files['parameters_file'], params)

In [ ]:
############################ RUN L2HMC ALGORITHM ############################
t0 = time.time()
for i in range(start_step, 1000):
    t1 = time.time()
    loss, samples, accept_prob, grads = train_one_iter(
        u1_dynamics,
        samples,
        optimizer,
        loss_fn=loss_fn,
        scale=loss_scale,
        metric=metric,
        eps=loss_eps,
        clip_value=clip_value,
        global_step=global_step
    )
    observables = np.array(u1_lattice.calc_plaq_observables(samples))
    _total_actions = observables[:, 0]
    _avg_plaquettes = observables[:, 1]
    _top_charges = observables[:, 2]
    
    total_actions.append(_total_actions)
    average_plaquettes.append(_avg_plaquettes)
    topological_charges.append(_top_charges)
    
    data = {
        'step': i,
        'global_step': global_step.numpy(),
        'loss': loss.numpy(),
        'time': time.time() - t1,
        'accept_prob': accept_prob.numpy().mean(),
        'eps': u1_dynamics.eps.numpy(),
        'avg_S': np.mean(_total_actions),
        'avg_top_charge': np.mean(_top_charges),
        'avg_plaq': np.mean(_avg_plaquettes),
        'avg_plaquettes': _avg_plaquettes,
        'top_charges': _top_charges,
        'total_actions': _total_actions,
        'avg_plaquettes_arr': np.array(average_plaquettes),
        'top_charges_arr': np.array(topological_charges),
        'total_actions_arr': np.array(total_actions),
        'samples': samples.numpy()
    }
    
    _ = write_run_data(run_files['run_info_file'], data, 'a')
    _ = print_run_data(data)
    
    if i % record_loss_every == 0:
        write_summaries(summary_writer, data)

    if i % save_steps == 0:
        save_run_data(checkpointer, log_dir, run_files, data)

In [ ]:
avg_plaqs_arr = np.array(average_plaquettes)
_avg_plaqs_arr = np.mean(avg_plaqs_arr, axis=0)
avg_plaq, avg_plaq_err = calc_avg_vals_errors(avg_plaqs_arr[-400:], num_blocks=10)
print(f'avg_plaq (mean from arr): {np.mean(_avg_plaqs_arr)}')
print(f'avg_plaq: {avg_plaq} +/- {avg_plaq_err}')

In [ ]:
calc_avg_vals_errors(_avg_plaqs_arr, num_blocks=50)

### Gradients check

Want to test that the value of the gradient ($\partial_x U(x)$) is the same when  
calculated using either:
1. Tensorflow's automatic differentiation (`tf.gradients`), or
2. The analytic expression (Eq. 17) from [reference](https://arxiv.org/pdf/hep-lat/9809160.pdf):
$$\partial_x U(x) = \beta \sum_{\nu \neq \mu} \sin \theta_{\mu\nu}(x - \hat\nu) - \sin \theta_{\mu\nu}(x)$$

In [ ]:
# fake data used to compare values of the gradient of the potential function
samples_ = tf.convert_to_tensor(
    [np.arange(128).reshape(u1_lattice.links.shape) for _ in range(4)],
    dtype=tf.float32
)

In [ ]:
# using tensorflow's automatic differentiation 
tf_grad = u1_dynamics.grad_potential(samples) 
# 276 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
# using the analytic expression
# (Eq. 17 from https://arxiv.org/pdf/hep-lat/9809160.pdf)
exact_grad = u1_lattice.grad_action(samples)
# 503 ms ± 33.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
print('exact grad:')
print(exact_grad[0].reshape(u1_lattice.links.shape)[:4, :4, 0])
print('\n')
print('tf grad:')
print(tf_grad[0].numpy().reshape(u1_lattice.links.shape)[:4, :4, 0])

In [ ]:
print('exact grad:')
print(exact_grad[0].reshape(u1_lattice.links.shape)[:4, :4, 1])
print('\n')
print('tf grad:')
print(tf_grad[0].numpy().reshape(u1_lattice.links.shape)[:4, :4, 1])

In [ ]:
################  tests  #############################
#u1_lattice._total_action()
#u1_lattice._average_plaquette()
#u1_lattice.total_action(u1_samples)
#u1_lattice.total_action()
#u1_lattice.average_plaquette(u1_samples)
#u1_lattice.average_plaquette()
#---------------------------------------------
#_momentum = tf.random_normal(tf.shape(u1_samples))
#_potential = np.array(u1_dynamics.potential(u1_samples_tensor))
#_kinetic = u1_dynamics.kinetic(_momentum)
#_grad_potential = u1_dynamics.grad_potential(u1_samples_tensor)
#print(_potential); print('\n')
#print(_kinetic); print('\n')
#print(_grad_potential)
#print(_kinetic.numpy()); print('\n')
#print(_hamiltonian.numpy()); print('\n')
#print(_grad_potential[0][:10])
#---------------------------------------------
#site = u1_lattice.get_random_site()
#u = np.random.randint(u1_lattice.dim)
#v = np.random.randint(u1_lattice.dim)
#plaq = u1_lattice.plaquette_operator(site, u, v)
#---------------------------------------------
#u1_lattice.total_action(u1_samples_tensor)
#u1_lattice.average_plaquette(u1_samples_tensor)
#transition_out = u1_dynamics.apply_transition(u1_samples_tensor)
#x_post, p_post, accept_prob, x_out = transition_out
#loss, x_out, x_accept_prob = l2hmc.compute_loss(u1_dynamics, u1_samples_tensor)
#x_accept_prob
###############################################

## Heatbath Algorithm

In [ ]:
time_size, space_size, dim, beta, num_samples = (16, 16, 2, 4., 5)
u1_lattice = GaugeLattice(time_size, space_size, dim, beta,
                          link_type='U1', num_samples=num_samples)
u1_samples = [sample.flatten() for sample in u1_lattice.samples]
u1_samples_tensor = tf.constant(np.stack(u1_samples), dtype=tf.float32)

eq_steps = 5000
acceptances = []
action_arr = [u1_lattice._total_action()]
avg_plaq_arr = [u1_lattice._average_plaquette()]
for i in range(eq_steps):
    action = u1_lattice._total_action()
    avg_plaq = u1_lattice._average_plaquette()
    change = avg_plaq - avg_plaq_arr[-1]
    avg_plaq_arr.append(avg_plaq)
    action_arr.append(action)
    print(f"Step: {i:<5g}\t action: {action:<8.4g}\t "
          f"avg plaq: {avg_plaq:<8.4g}\t change: {change:<8.4g}")
    accept = 0
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            accept += u1_lattice._update_link(site, d)
    acceptances.append(accept)
# 12.2s for 500 equilibration steps

avg_plaq_arr = [0]
p = k - j
for k in range(j, 40000):
    avg_plaq = u1_lattice._average_plaquette()
    change = avg_plaq - avg_plaq_arr[p-1]
    avg_plaq_arr.append(avg_plaq)
    print(f"Step: {k:<5g}: avg plaq: {avg_plaq:>12.4g} change: {change:12.4g}")
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            _ = u1_lattice._update_link(site, d)

num_acceptances = 0
measure_steps = 10000
avg_plq = np.zeros(measure_steps)
for step in range(measure_steps):
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            num_acceptances += u1_lattice._update_link(site, d)
    avg_plq[step] = u1_lattice._average_plaquette()

## OLD

### GMM Model (test L2HMC)

In [ ]:
sigmas, distribution = gen_ring(1., var=0.02, nb_mixtures=4)

gmm_potential = distribution.get_energy_function()
gmm_dynamics = _l2hmc.Dynamics(x_dim=2, minus_loglikelihood_fn=gmm_potential,
                               n_steps=25, eps=0.1)

samples = distribution.get_samples(200)

_position = tf.convert_to_tensor(samples, dtype=tf.float32)
_momentum = tf.random_normal(tf.shape(_position))
_hamiltonian = gmm_dynamics.hamiltonian(_position, _momentum)

grad_pot = gmm_dynamics.grad_potential(_position, _momentum)

grad_pot.shape

gmm_dynamics.masks[0]

scale, translation, transformed = gmm_dynamics.position_fn([_position, _momentum, gmm_dynamics._get_time(0)])

scale.shape

_position.shape

gmm_dynamics.masks[0].shape

gmm_dynamics.masks[0] * _position

### Construct GaugeLattice with SU(3) gauge group

In [ ]:
time_size = 2
space_size = 4
dim = 4
beta = 1.
link_type = 'SU3' 
batch_size = 3
gauge_lattice = GaugeLattice(time_size, space_size, dim, beta, link_type)
# create `num_samples` random samples of GaugeLattice.links
links_samples = gauge_lattice.get_links_samples(batch_size, link_type=link_type)

In [ ]:
gauge_energy_fn = gauge_lattice.get_energy_function()
gauge_dynamics = l2hmc.GaugeDynamics(gauge_lattice, 
                                     minus_loglikelihood_fn=gauge_energy_fn, 
                                     batch_size=3, n_steps=5, eps=0.1)

In [ ]:
gauge_lattice.links.shape

In [ ]:
potential_arr = gauge_dynamics.potential(links_samples, batch_size)

[i.numpy() for i in potential_arr]

In [ ]:
_momentum = tf.random_normal(tf.shape(links_samples))

In [ ]:
gauge_dynamics.kinetic(_momentum).numpy()

In [ ]:
_x = links_samples
#_momentum = tf.random_normal(tf.shape(_x))
_hamiltonian = gauge_dynamics.hamiltonian(_x, _momentum)
_hamiltonian

### Construct IsingLattice

In [ ]:
ising_batch_size = 10
ising_lattice = IsingLattice(3, 4)
ising_samples = [ising_lattice._randomize() for _ in range(ising_batch_size)]

In [ ]:
ising_energy_fn = ising_lattice.get_energy_function()
ising_dynamics = l2hmc.LatticeDynamics(ising_lattice, 
                                       minus_loglikelihood_fn=ising_energy_fn,
                                       batch_size=ising_batch_size, 
                                       n_steps=10, eps=0.1)
#dynamics = l2hmc.LDynamics(latt.sites.shape, minus_loglikelihood_fn=energy_fn, n_steps=10, eps=0.1)

In [ ]:
ising_dynamics.potential(samples, batch_size)

In [ ]:
_iposition = ising_samples
_imomentum = tf.random_normal(tf.shape(_iposition))
_ihamiltonian = dynamics.hamiltonian(_iposition, _imomentum)
_ihamiltonian

In [ ]:
_isample = _iposition[0].reshape(ising_lattice.num_sites)
#dynamics.grad_potential(np.array(_position).reshape(-1, lattice.num_sites))

In [ ]:
grad_pot = dynamics.grad_potential(ising_samples)

In [ ]:
ising_jacobian = jacobian(dynamics.potential, ising_samples)

In [ ]:
grad_fn = tfe.gradients_function(lattice._calc_energy, params=[0])

In [ ]:
_jacobian = jacobian(dynamics.potential, _position)

In [ ]:
lattice.calc_energy(_position, batch_size)

In [ ]:
#dynamics.position_fn(momentum, latt.sites.flatten()[:], dynamics)
#dynamics._forward_lf(latt.sites.flatten()[:], momentum, 0)
dynamics._forward_lf(np.array(_position).reshape(-1, lattice.num_sites),
                     np.array(_momentum).reshape(-1, lattice.num_sites), 1)